In [21]:
from pprint import pprint as pp
import numpy as np
import cv2
import random as rnd

In [24]:
class Line():
  def __init__(self, index, top, down) -> None:
    self.index = index
    self.down = down
    self.top = top

  def __getitem__(self, key):
    if key == 0: return self.index
    if key == 1: return self.top
    if key == 2: return self.down
    
  def __repr__(self):
    return ("{ index: " + str(self.index) + ", " +\
            "top: " + str(self.top) + ", "+ \
            "down: " + str(self.down) + "}")

def get_lines(mask_inv):
    graph = []

    for j in range(mask_inv.shape[1]):
        higher = 0
        lower = 0

        cntBlank = 0
        lines = []

        for i in range(mask_inv.shape[0]):
            if mask_inv[i,j] == 255: 
                cntBlank += 1
                if lower == 0:
                    continue
            
            if lower == 0: lower = i
            if mask_inv[i,j] != 255: 
                higher = i
                cntBlank = 0
            if cntBlank > 10:
                lines.append(Line(j, higher, lower))
                lower = 0
                higher = 0
        # if len(lines) > 0:
        for l in lines:
            graph.append(l)
        lines.clear()
    
    return graph

def get_mask(img):
    # Convert BGR to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # define range of black color in HSV
    lower_val = np.array([0,0,0])
    upper_val = np.array([50,100,100])

    # Threshold the HSV image to get only black colors
    mask = cv2.inRange(hsv, lower_val, upper_val)

    # invert mask to get black symbols on white background
    mask_inv = cv2.bitwise_not(mask)

    return mask_inv

def draw_islands(ecg):
    test_img = np.ones((
        mask_inv.shape[0],
        mask_inv.shape[1],
        3
        )) * 255

    for island in ecg:
        for i in island:
            clr = (
                rnd.randint(0, 255),
                rnd.randint(0, 255),
                rnd.randint(0, 255)
            )
            cv2.line(
                test_img, 
                (i.index, i.top),
                (i.index, i.down),
                clr
                )
    cv2.imwrite(r'../images/islands.png',test_img)


In [25]:
img = cv2.imread(r"../images/input.jpg")
mask_inv = get_mask(img)
graph = get_lines(mask_inv)

ecg = []

while (len(graph) != 0):
    temp = [graph.pop(0)]
    for i, k in enumerate(graph):
        
        if (temp[-1].index != k.index-1):
            # если эта точка не сосед предыдущей
            continue
        
        if (temp[-1].top > k.down or k.top > temp[-1].down):
            # если эта точка не сосед предыдущей
            continue
            
        # если эта точка все таки сосед предыдущей
        temp.append(graph.pop(i))  
        
    if (len(temp) != 0):
        ecg.append(temp)
        # print(temp)
        # print(">__________________________<")

draw_islands(ecg)

In [27]:
blank3 = np.ones(img.shape, np.uint8)*255

prev = graph[0]
lastAcc = prev[0]

prevPoint = 0
mainPoints = []
mainPoints.append(graph[0].top)
prevPoint = graph[0].down
status = False
print("Я начну с точки ", mainPoints[0])
clr = (147, 20, 255)
img[mainPoints[0]][graph[0].index] = clr

for p in graph[1:]:
    
    clr = 125*(p[0]%2)
    
    if (p[0]%2 == 0):
        clr = (255, 255, 0)
    else:
        clr = (255, 0, 0)
        
    if(status == False):
        if(mainPoints[-1] < p.top):
            if(prevPoint <= p.down):
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
            else:
                if((p.top - mainPoints[-1]) > (p.down - prevPoint)):
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
                elif((p.top - mainPoints[-1]) < (p.down - prevPoint)):
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
                else:
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
        elif(mainPoints[-1] > p.top):
            if(prevPoint >= p.down):
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
            else:
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
        else:
            if(prevPoint <= p.down):
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
            else:
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
    else:
        if(mainPoints[-1] > p.down):
            if(prevPoint >= p.top):
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
            else:
                if((p.down - mainPoints[-1]) < (p.top - prevPoint)):
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
                elif((p.down - mainPoints[-1]) > (p.top - prevPoint)):
                    status = False
                    mainPoints.append(p.top)
                    prevPoint = p.down
                else:
                    status = True
                    mainPoints.append(p.down)
                    prevPoint = p.top
        elif(mainPoints[-1] < p.down):
            status = False
            mainPoints.append(p.top)
            prevPoint = p.down
        else:
            if(p.top > prevPoint):
                status = False
                mainPoints.append(p.top)
                prevPoint = p.down
            else:
                status = True
                mainPoints.append(p.down)
                prevPoint = p.top
    
    cv2.line(img, (p[0],p[2]), (p[0], p[1]), clr)
    cv2.line(blank3, (p[0],p[2]), (p[0], p[1]), clr)
    if(status == True):
        clr = (0, 215, 255)
    else:
        clr = (147, 20, 255)
    img[mainPoints[-1]][p.index] = clr


Я начну с точки  562


In [28]:
cv2.imwrite(r'../images/mask_inv.png',mask_inv)
cv2.imwrite(r'../images/output2.png',img)
cv2.imwrite(r'../images/output.png',blank3)

True